# 서울시 교통사고 시각화

In [1]:
import pandas as pd

In [3]:
traffic_Seoul = pd.read_csv('data/newSeoul_2005_2019.csv', encoding='utf-8')
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [4]:
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [10]:
# index 0번부터
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [17]:
# 자치구별 별 합계
traffic_analysis = traffic_Seoul_2019.groupby('자치구명').sum()
# '년도', '월' column 삭제
traffic_analysis.drop(['년도', '월'], axis=1, inplace=True)
# 컬럼 순서 '발생건수', '부상자수', '사망자수' 순으로 변경
traffic_analysis = traffic_analysis[['발생건수', '부상자수', '사망자수']]
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [20]:
traffic_analysis2 = pd.pivot_table(traffic_Seoul_2019[['자치구명','발생건수', '부상자수', '사망자수']], index='자치구명', aggfunc=sum)
traffic_analysis2.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [25]:
seoul_limit = pd.read_csv('data/seoul.csv', encoding='euc-kr')
seoul_limit.columns = ['자치구명', 'lon', 'lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [26]:
# merge
data_result = pd.merge(traffic_analysis, seoul_limit, on='자치구명')

In [27]:
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [28]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [29]:
# int를 float로 변환
data_result[['발생건수', '부상자수', '사망자수']] = data_result[['발생건수', '부상자수', '사망자수']].astype(float)

In [30]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


In [31]:
import folium

In [70]:
# 지도 생성
map_발생 = folium.Map(location=[37.5502, 126.982], zoom_start=12)
# 마커 표시
for i in range(len(data_result)):
  html = '<h4>' + data_result.iloc[i]['자치구명'] + '</h4>'
  html += '<p>발생건수: ' + str(data_result.iloc[i]['발생건수']) + '</p>'
  html += '<p>부상자수: ' + str(data_result.iloc[i]['부상자수']) + '</p>'
  html += '<p>사망자수: ' + str(data_result.iloc[i]['사망자수']) + '</p>'
  popup = folium.Popup(html, min_width=100, max_width=100)
  # marker circle size = 발생건수
  folium.CircleMarker(location=[data_result.iloc[i]['lat'], data_result.iloc[i]['lon']], radius=data_result.iloc[i]['발생건수']*0.01, color='blue', fill=True).add_to(map_발생)
  # normal marker
  folium.Marker(location=[data_result.iloc[i]['lat'], data_result.iloc[i]['lon']], popup=popup).add_to(map_발생)
# 지도 출력
map_발생

In [77]:
map_사망 = folium.Map(location=[37.5502, 126.982], zoom_start=12)
# data_result sort by 사망자수
data_result = data_result.sort_values(by='사망자수', ascending=False)

# 마커 표시
for i in range(len(data_result)):
  html = '<h4>' + data_result.iloc[i]['자치구명'] + '</h4>'
  html += '<p>발생건수: ' + str(data_result.iloc[i]['발생건수']) + '</p>'
  html += '<p>부상자수: ' + str(data_result.iloc[i]['부상자수']) + '</p>'
  html += '<p>사망자수: ' + str(data_result.iloc[i]['사망자수']) + '</p>'
  popup = folium.Popup(html, min_width=100, max_width=100)
  # normal marker
  folium.Marker(location=[data_result.iloc[i]['lat'], data_result.iloc[i]['lon']], popup=popup).add_to(map_사망)

folium.CircleMarker(location=[data_result.iloc[0]['lat'], data_result.iloc[0]['lon']], radius=data_result.iloc[0]['사망자수']*2, color='red', fill=True).add_to(map_사망)
folium.CircleMarker(location=[data_result.iloc[len(data_result) - 1]['lat'], data_result.iloc[len(data_result) - 1]['lon']], radius=data_result.iloc[len(data_result) - 1]['사망자수']*2, color='red', fill=True).add_to(map_사망)
# 지도 출력
map_사망